In [1]:
# Mexico SPFEAS using Sentinel
import sys, os, inspect
import pandas as pd
import geopandas as gpd

from gbdxtools import Interface
from gbdxtools import CatalogImage
from shapely.geometry import box

cmd_folder = os.path.dirname(os.getcwd())
if cmd_folder not in sys.path:
    sys.path.insert(0, cmd_folder)

from GOST_GBDx_Tools import gbdxTasks
from GOST_GBDx_Tools import gbdxURL_misc
from GOST_GBDx_Tools import imagery_search

#In order for the interface to be properly authenticated, follow instructions here:
#   http://gbdxtools.readthedocs.io/en/latest/user_guide.html
#   For Ben, the .gbdx-config file belongs in C:\Users\WB411133 (CAUSE no one else qill f%*$&ing tell you that)
gbdx = Interface()
gbdx.s3.info
curTasks = gbdxTasks.GOSTTasks(gbdx)
gbdxUrl = gbdxURL_misc.gbdxURL(gbdx)

In [55]:
# Define input datasets
AOI = r"Q:\WORKINGPROJECTS\Mexico_Poverty\agebs\urban_agebs_Buffer_200m_Diss.shp"
imageryCSV = AOI.replace(".shp", "_imagerySearch.csv")
### I don't know why these are required for the imagery search - I don't think they do anything, but it works, so just leave it
outFolder = "C:/temp"
searchName = "FUBAR"

cutOffDate = "1-Jan-18"  #earliest acceptable date
optimalDate = "5-Mar-19" #best date
maxAcceptableCloud = 100
curoffNadir = 90

inS = gpd.read_file(AOI)
inS = inS.to_crs({'init': u'epsg:4326'})
inS['SEN1_id'] = ''
for idx, row in inS.iterrows():
    curRes_S1 = gbdx.vectors.query(str(row['geometry']), query="item_type:SENTINEL1")
    xx = [r['properties']['attributes'] for r in curRes_S1]
    s1Results = pd.DataFrame(xx)
    s1Results = s1Results.sort_values(by='timestamp')
    s1Results['P'] = s1Results['polarizations'].apply(lambda x: ','.join(x))
    s1Results = s1Results[s1Results['P'] == 'VH,VV']        
    inS['SEN1_id'][idx] = s1Results.iloc[s1Results.shape[0] - 1]['id']
    


C:\WBG\Anaconda\envs\gbdxtools\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [56]:
inS

,Id,geometry,SEN1_id
0,0,"POLYGON ((-92.28227767480703 14.8787604139104,...",S1B_IW_GRDH_1SDV_20181228T002156_20181228T0022...
1,0,"POLYGON ((-92.29890072116645 14.8858976439897,...",S1B_IW_GRDH_1SDV_20181228T002156_20181228T0022...
2,0,POLYGON ((-92.27119494940874 14.91499330544682...,S1B_IW_GRDH_1SDV_20181228T002156_20181228T0022...
3,0,POLYGON ((-92.24217202215144 14.92062123612466...,S1B_IW_GRDH_1SDV_20181228T002156_20181228T0022...
4,0,POLYGON ((-92.25132706730736 14.90956884333488...,S1B_IW_GRDH_1SDV_20181228T002156_20181228T0022...
5,0,POLYGON ((-92.26249287145932 14.93775259674631...,S1B_IW_GRDH_1SDV_20181228T002156_20181228T0022...
6,0,POLYGON ((-92.25135453417161 14.94230872829409...,S1B_IW_GRDH_1SDV_20181228T002156_20181228T0022...
7,0,POLYGON ((-92.07959262856482 15.06052295188652...,S1A_IW_GRDH_1SDV_20181229T001420_20181229T0014...
8,0,"POLYGON ((-92.45719609479019 15.1302217692843,...",S1B_IW_GRDH_1SDV_20181228T002156_20181228T0022...
9,0,POLYGON ((-92.64781204361637 15.32933397781185...,S1B_IW_GRDH_1SDV_20181228T002156_20181228T0022...


In [7]:
# Read in the radar data and write some data to file in order to process
# https://notebooks.geobigdata.io/hub/notebooks/05c0uyupni3g9c862lgy?tab=code

from gbdxtools.images.rda_image import RDAImage
from gbdxtools.rda.interface import RDA

from shapely.geometry import box,shape

gbdx = Interface()
rda = RDA()

def search(_type, bbox):
    aoi = box(*bbox)
    query = 'item_type:{}'.format(_type)
    return gbdx.vectors.query(aoi.wkt, query, count=10)

class Sentinel1Image(RDAImage):
    def __new__(cls, cat_id, polarization="VH", **kwargs):
        img = sentinel = rda.Sentinel1Read(SentinelId=cat_id,
                                           sentinel1Polarization=polarization)
        self = super(Sentinel1Image, cls).__new__(cls, img)
        self.cat_id = cat_id
        return self            

In [ ]:
baseFolder = r"Q:\WORKINGPROJECTS\Mexico_Poverty\RawImagery\Sentinel"
for idx, row in inS.iterrows():
    catID = row['SEN1_id']
    outFolder = os.path.join(baseFolder, "AGEB_%s" % idx)
    if not os.path.exists(outFolder):
        os.makedirs(outFolder)
    vvFile = os.path.join(outFolder, "VV___%s.tif" % catID)
    vhFile = os.path.join(outFolder, "VH___%s.tif" % catID)
    if not os.path.exists(vvFile):
        try:
            sentVV = Sentinel1Image(catID,polarization="VV")
            xx_vv = sentVV.aoi(wkt=str(inS['geometry'][0]))
            xx_vv.geotiff(path=vvFile)
        except:
            print("Error processing VV for %s: %s" % (idx, catID))
    if not os.path.exists(vhFile):
        try:
            sentVH = Sentinel1Image(catID,polarization="VH")        
            xx_vh = sentVH.aoi(wkt=str(inS['geometry'][0]))
            xx_vh.geotiff(path=vhFile)
        except:
            print("Error processing VH for %s: %s" % (idx, catID))
            
    print("Processed %s" % idx)

Processed 0
Processed 1
Processed 2
Processed 3
Processed 4
Processed 5
Processed 6
Error processing VV for 7: S1A_IW_GRDH_1SDV_20181229T001420_20181229T001445_025233_02CA25_B820
Error processing VH for 7: S1A_IW_GRDH_1SDV_20181229T001420_20181229T001445_025233_02CA25_B820
Processed 7
Processed 8
Processed 9
Processed 10
Error processing VV for 11: S1B_IW_GRDH_1SDV_20181230T121800_20181230T121817_014271_01A8B8_2F86
Error processing VH for 11: S1B_IW_GRDH_1SDV_20181230T121800_20181230T121817_014271_01A8B8_2F86
Processed 11
Error processing VV for 12: S1A_IW_GRDH_1SDV_20181226T120155_20181226T120220_025196_02C8CB_1FF8
Error processing VH for 12: S1A_IW_GRDH_1SDV_20181226T120155_20181226T120220_025196_02C8CB_1FF8
Processed 12
Processed 13
Error processing VV for 14: S1A_IW_GRDH_1SDV_20181231T121015_20181231T121043_025269_02CB65_DA3C
Error processing VH for 14: S1A_IW_GRDH_1SDV_20181231T121015_20181231T121043_025269_02CB65_DA3C
Processed 14
Error processing VV for 15: S1A_IW_GRDH_1SDV_2018

Error processing VH for 50: S1A_IW_GRDH_1SDV_20181226T120155_20181226T120220_025196_02C8CB_1FF8
Processed 50
Error processing VV for 51: S1A_IW_GRDH_1SDV_20181226T120155_20181226T120220_025196_02C8CB_1FF8
Error processing VH for 51: S1A_IW_GRDH_1SDV_20181226T120155_20181226T120220_025196_02C8CB_1FF8
Processed 51
Error processing VV for 52: S1A_IW_GRDH_1SDV_20181226T120155_20181226T120220_025196_02C8CB_1FF8
Error processing VH for 52: S1A_IW_GRDH_1SDV_20181226T120155_20181226T120220_025196_02C8CB_1FF8
Processed 52
Error processing VV for 53: S1A_IW_GRDH_1SDV_20181226T120155_20181226T120220_025196_02C8CB_1FF8
Error processing VH for 53: S1A_IW_GRDH_1SDV_20181226T120155_20181226T120220_025196_02C8CB_1FF8
Processed 53
Error processing VV for 54: S1A_IW_GRDH_1SDV_20181226T120155_20181226T120220_025196_02C8CB_1FF8
Error processing VH for 54: S1A_IW_GRDH_1SDV_20181226T120155_20181226T120220_025196_02C8CB_1FF8
Processed 54
Error processing VV for 55: S1A_IW_GRDH_1SDV_20181226T120155_20181226T1

Error processing VH for 90: S1B_IW_GRDH_1SDV_20181230T121735_20181230T121800_014271_01A8B8_0D68
Processed 90
Error processing VV for 91: S1B_IW_GRDH_1SDV_20181230T121735_20181230T121800_014271_01A8B8_0D68
Error processing VH for 91: S1B_IW_GRDH_1SDV_20181230T121735_20181230T121800_014271_01A8B8_0D68
Processed 91
Error processing VV for 92: S1A_IW_GRDH_1SDV_20181226T120155_20181226T120220_025196_02C8CB_1FF8
Error processing VH for 92: S1A_IW_GRDH_1SDV_20181226T120155_20181226T120220_025196_02C8CB_1FF8
Processed 92
Error processing VV for 93: S1B_IW_GRDH_1SDV_20181230T121735_20181230T121800_014271_01A8B8_0D68
Error processing VH for 93: S1B_IW_GRDH_1SDV_20181230T121735_20181230T121800_014271_01A8B8_0D68
Processed 93
Error processing VV for 94: S1B_IW_GRDH_1SDV_20181230T121735_20181230T121800_014271_01A8B8_0D68
Error processing VH for 94: S1B_IW_GRDH_1SDV_20181230T121735_20181230T121800_014271_01A8B8_0D68
Processed 94
Error processing VV for 95: S1B_IW_GRDH_1SDV_20181230T121735_20181230T1

Error processing VV for 130: S1A_IW_GRDH_1SDV_20181230T005616_20181230T005641_025248_02CAA3_2E7A
Error processing VH for 130: S1A_IW_GRDH_1SDV_20181230T005616_20181230T005641_025248_02CAA3_2E7A
Processed 130
Error processing VV for 131: S1A_IW_GRDH_1SDV_20181231T120946_20181231T121015_025269_02CB65_5163
Error processing VH for 131: S1A_IW_GRDH_1SDV_20181231T120946_20181231T121015_025269_02CB65_5163
Processed 131
Error processing VV for 132: S1A_IW_GRDH_1SDV_20181230T005616_20181230T005641_025248_02CAA3_2E7A
Error processing VH for 132: S1A_IW_GRDH_1SDV_20181230T005616_20181230T005641_025248_02CAA3_2E7A
Processed 132
Error processing VV for 133: S1A_IW_GRDH_1SDV_20181226T120126_20181226T120155_025196_02C8CB_B775
Error processing VH for 133: S1A_IW_GRDH_1SDV_20181226T120126_20181226T120155_025196_02C8CB_B775
Processed 133
Error processing VV for 134: S1A_IW_GRDH_1SDV_20181231T120946_20181231T121015_025269_02CB65_5163
Error processing VH for 134: S1A_IW_GRDH_1SDV_20181231T120946_20181231T

Error processing VH for 169: S1B_IW_GRDH_1SDV_20181230T121710_20181230T121735_014271_01A8B8_56AB
Processed 169
Error processing VV for 170: S1B_IW_GRDH_1SDV_20181231T004719_20181231T004744_014279_01A8FA_1B13
Error processing VH for 170: S1B_IW_GRDH_1SDV_20181231T004719_20181231T004744_014279_01A8FA_1B13
Processed 170
Error processing VV for 171: S1A_IW_GRDH_1SDV_20181231T120946_20181231T121015_025269_02CB65_5163
Error processing VH for 171: S1A_IW_GRDH_1SDV_20181231T120946_20181231T121015_025269_02CB65_5163
Processed 171
Error processing VV for 172: S1B_IW_GRDH_1SDV_20181231T004719_20181231T004744_014279_01A8FA_1B13
Error processing VH for 172: S1B_IW_GRDH_1SDV_20181231T004719_20181231T004744_014279_01A8FA_1B13
Processed 172
Error processing VV for 173: S1B_IW_GRDH_1SDV_20181231T004719_20181231T004744_014279_01A8FA_1B13
Error processing VH for 173: S1B_IW_GRDH_1SDV_20181231T004719_20181231T004744_014279_01A8FA_1B13
Processed 173
Error processing VV for 174: S1B_IW_GRDH_1SDV_20181231T00

...

Error processing VV for 604: S1B_IW_GRDH_1SDV_20181231T004744_20181231T004809_014279_01A8FA_BC0A
Error processing VH for 604: S1B_IW_GRDH_1SDV_20181231T004744_20181231T004809_014279_01A8FA_BC0A
Processed 604
Error processing VV for 605: S1B_IW_GRDH_1SDV_20181231T004744_20181231T004809_014279_01A8FA_BC0A
Error processing VH for 605: S1B_IW_GRDH_1SDV_20181231T004744_20181231T004809_014279_01A8FA_BC0A
Processed 605
Error processing VV for 606: S1B_IW_GRDH_1SDV_20181231T004744_20181231T004809_014279_01A8FA_BC0A
Error processing VH for 606: S1B_IW_GRDH_1SDV_20181231T004744_20181231T004809_014279_01A8FA_BC0A
Processed 606
Error processing VV for 607: S1B_IW_GRDH_1SDV_20181230T121641_20181230T121710_014271_01A8B8_D12A
Error processing VH for 607: S1B_IW_GRDH_1SDV_20181230T121641_20181230T121710_014271_01A8B8_D12A
Processed 607
Error processing VV for 608: S1A_IW_GRDH_1SDV_20181230T235914_20181230T235939_025262_02CB2C_B307
Error processing VH for 608: S1A_IW_GRDH_1SDV_20181230T235914_20181230T

Error processing VH for 643: S1A_IW_GRDH_1SDV_20181230T005641_20181230T005706_025248_02CAA3_312D
Processed 643
Error processing VV for 644: S1B_IW_GRDH_1SDV_20181231T004744_20181231T004809_014279_01A8FA_BC0A
Error processing VH for 644: S1B_IW_GRDH_1SDV_20181231T004744_20181231T004809_014279_01A8FA_BC0A
Processed 644
Error processing VV for 645: S1B_IW_GRDH_1SDV_20181231T004744_20181231T004809_014279_01A8FA_BC0A
Error processing VH for 645: S1B_IW_GRDH_1SDV_20181231T004744_20181231T004809_014279_01A8FA_BC0A
Processed 645
Error processing VV for 646: S1B_IW_GRDH_1SDV_20181231T004744_20181231T004809_014279_01A8FA_BC0A
Error processing VH for 646: S1B_IW_GRDH_1SDV_20181231T004744_20181231T004809_014279_01A8FA_BC0A
Processed 646
Error processing VV for 647: S1B_IW_GRDH_1SDV_20181231T004744_20181231T004809_014279_01A8FA_BC0A
Error processing VH for 647: S1B_IW_GRDH_1SDV_20181231T004744_20181231T004809_014279_01A8FA_BC0A
Processed 647
Error processing VV for 648: S1B_IW_GRDH_1SDV_20181231T00

Error processing VV for 683: S1B_IW_GRDH_1SDV_20181231T004744_20181231T004809_014279_01A8FA_BC0A
Error processing VH for 683: S1B_IW_GRDH_1SDV_20181231T004744_20181231T004809_014279_01A8FA_BC0A
Processed 683
Error processing VV for 684: S1B_IW_GRDH_1SDV_20181231T004744_20181231T004809_014279_01A8FA_BC0A
Error processing VH for 684: S1B_IW_GRDH_1SDV_20181231T004744_20181231T004809_014279_01A8FA_BC0A
Processed 684
Error processing VV for 685: S1B_IW_GRDH_1SDV_20181231T004744_20181231T004809_014279_01A8FA_BC0A
Error processing VH for 685: S1B_IW_GRDH_1SDV_20181231T004744_20181231T004809_014279_01A8FA_BC0A
Processed 685
Error processing VV for 686: S1B_IW_GRDH_1SDV_20181227T115219_20181227T115248_014227_01A732_9ED0
Error processing VH for 686: S1B_IW_GRDH_1SDV_20181227T115219_20181227T115248_014227_01A732_9ED0
Processed 686
Error processing VV for 687: S1B_IW_GRDH_1SDV_20181231T004744_20181231T004809_014279_01A8FA_BC0A
Error processing VH for 687: S1B_IW_GRDH_1SDV_20181231T004744_20181231T

Error processing VH for 722: S1B_IW_GRDH_1SDV_20181230T121641_20181230T121710_014271_01A8B8_D12A
Processed 722
Error processing VV for 723: S1B_IW_GRDH_1SDV_20181226T003938_20181226T004003_014206_01A67F_FF5A
Error processing VH for 723: S1B_IW_GRDH_1SDV_20181226T003938_20181226T004003_014206_01A67F_FF5A
Processed 723
Error processing VV for 724: S1A_IW_GRDH_1SDV_20181230T235914_20181230T235939_025262_02CB2C_B307
Error processing VH for 724: S1A_IW_GRDH_1SDV_20181230T235914_20181230T235939_025262_02CB2C_B307
Processed 724
Error processing VV for 725: S1B_IW_GRDH_1SDV_20181226T003938_20181226T004003_014206_01A67F_FF5A
Error processing VH for 725: S1B_IW_GRDH_1SDV_20181226T003938_20181226T004003_014206_01A67F_FF5A
Processed 725
Error processing VV for 726: S1B_IW_GRDH_1SDV_20181226T003938_20181226T004003_014206_01A67F_FF5A
Error processing VH for 726: S1B_IW_GRDH_1SDV_20181226T003938_20181226T004003_014206_01A67F_FF5A
Processed 726
Error processing VV for 727: S1B_IW_GRDH_1SDV_20181226T00

Error processing VV for 762: S1B_IW_GRDH_1SDV_20181228T123257_20181228T123326_014242_01A7B5_5114
Error processing VH for 762: S1B_IW_GRDH_1SDV_20181228T123257_20181228T123326_014242_01A7B5_5114
Processed 762
Error processing VV for 763: S1A_IW_GRDH_1SDV_20181230T005706_20181230T005731_025248_02CAA3_77BB
Error processing VH for 763: S1A_IW_GRDH_1SDV_20181230T005706_20181230T005731_025248_02CAA3_77BB
Processed 763
Error processing VV for 764: S1A_IW_GRDH_1SDV_20181230T235914_20181230T235939_025262_02CB2C_B307
Error processing VH for 764: S1A_IW_GRDH_1SDV_20181230T235914_20181230T235939_025262_02CB2C_B307
Processed 764
Error processing VV for 765: S1A_IW_GRDH_1SDV_20181230T005706_20181230T005731_025248_02CAA3_77BB
Error processing VH for 765: S1A_IW_GRDH_1SDV_20181230T005706_20181230T005731_025248_02CAA3_77BB
Processed 765
Error processing VV for 766: S1B_IW_GRDH_1SDV_20181228T123257_20181228T123326_014242_01A7B5_5114
Error processing VH for 766: S1B_IW_GRDH_1SDV_20181228T123257_20181228T

'S1B_IW_GRDH_1SDV_20181228T002156_20181228T002221_014235_01A777_234E'

'C:/temp/vh.tif'